In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import re

In [25]:
data_1 = pd.read_csv("log_week_end.csv")
data_1.head()

,rn,loguser,query
0,2530,etl_2048,"from tbl_157224,join tbl_157223"
1,5658,etl_2048,"from tbl_195789,join tbl_195791"
2,14087,etl_2048,"from tbl_144504,join tbl_144505"
3,1272,etl_1151,"JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,f..."
4,9974,etl_1151,"from tbl_8053,join tbl_9874,join tbl_10303,int..."


In [26]:
data_1.shape

(641138, 3)

In [27]:
data_2 = pd.read_csv("log_week_2_end.csv")
data_2.head()

,rn,loguser,query,duration
0,16493642,etl_528,from tbl_111329,duration: 240.851 ms
1,12761109,etl_528,from tbl_127353,duration: 2165.935 ms
2,28294808,etl_528,from tbl_128986,duration: 17661.026 ms
3,12703938,etl_528,from tbl_155188,duration: 1047.568 ms
4,5906380,etl_528,from tbl_48128,duration: 171.001 ms


In [28]:
data_2.shape

(630078, 4)

In [29]:
data_2.drop(columns=["duration"], inplace=True)
data_2.head()

,rn,loguser,query
0,16493642,etl_528,from tbl_111329
1,12761109,etl_528,from tbl_127353
2,28294808,etl_528,from tbl_128986
3,12703938,etl_528,from tbl_155188
4,5906380,etl_528,from tbl_48128


In [30]:
data = pd.concat([data_1, data_2])
data.head()

,rn,loguser,query
0,2530,etl_2048,"from tbl_157224,join tbl_157223"
1,5658,etl_2048,"from tbl_195789,join tbl_195791"
2,14087,etl_2048,"from tbl_144504,join tbl_144505"
3,1272,etl_1151,"JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,f..."
4,9974,etl_1151,"from tbl_8053,join tbl_9874,join tbl_10303,int..."


In [31]:
data.shape

(1271216, 3)

In [32]:
data.shape[0] == data_1.shape[0] + data_2.shape[0]

True

In [33]:
data.rn.value_counts()

219209      2
635187      2
1009786     2
851330      2
720487      2
           ..
500104      1
425081      1
483452      1
396872      1
21023432    1
Name: rn, Length: 1237478, dtype: int64

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271216 entries, 0 to 630077
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   rn       1271216 non-null  int64 
 1   loguser  1271216 non-null  object
 2   query    1271216 non-null  object
dtypes: int64(1), object(2)
memory usage: 38.8+ MB


In [35]:
data[data.rn == 219209]

,rn,loguser,query
391310,219209,etl_1151,from tbl_8398
288158,219209,etl_2048,from tbl_166759


In [36]:
data[~data.loguser.str.startswith("etl")]

,rn,loguser,query
5,1018,dev_473,"INTO tbl_7094,INTO tbl_7095,from tbl_7095,from..."
13,1361,dev_473,"join tbl_33332,join tbl_151403,INTO tbl_385661"
16,7032,dev_473,"INTO tbl_7097,FROM tbl_7097,FROM tbl_7097,INTO..."
19,12839,dev_359,"join tbl_112511,join tbl_112583,join tbl_11270..."
32,3944,dev_473,"INTO tbl_7097,FROM tbl_7097,FROM tbl_7097,INTO..."
...,...,...,...
630027,11806240,dev_332,"from tbl_44939,from tbl_44939,from tbl_211427,..."
630038,22243557,dev_1586,into tbl_24696
630040,3100643,dev_359,FROM tbl_42632
630041,8606742,dev_359,FROM tbl_33186


In [37]:
"""
Добавление нового столбца, типа пользователя. Если пользователь etl - загрузчик данных, тогда 1
Если программиста dev - 0
"""
data["type_user"] = data.loguser.apply(lambda x: 1 if x.startswith("etl") else 0)

In [38]:
data.head()

,rn,loguser,query,type_user
0,2530,etl_2048,"from tbl_157224,join tbl_157223",1
1,5658,etl_2048,"from tbl_195789,join tbl_195791",1
2,14087,etl_2048,"from tbl_144504,join tbl_144505",1
3,1272,etl_1151,"JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,f...",1
4,9974,etl_1151,"from tbl_8053,join tbl_9874,join tbl_10303,int...",1


In [39]:
data[data.type_user == 0]

,rn,loguser,query,type_user
5,1018,dev_473,"INTO tbl_7094,INTO tbl_7095,from tbl_7095,from...",0
13,1361,dev_473,"join tbl_33332,join tbl_151403,INTO tbl_385661",0
16,7032,dev_473,"INTO tbl_7097,FROM tbl_7097,FROM tbl_7097,INTO...",0
19,12839,dev_359,"join tbl_112511,join tbl_112583,join tbl_11270...",0
32,3944,dev_473,"INTO tbl_7097,FROM tbl_7097,FROM tbl_7097,INTO...",0
...,...,...,...,...
630027,11806240,dev_332,"from tbl_44939,from tbl_44939,from tbl_211427,...",0
630038,22243557,dev_1586,into tbl_24696,0
630040,3100643,dev_359,FROM tbl_42632,0
630041,8606742,dev_359,FROM tbl_33186,0


In [40]:
string = "from tbl_44939,from tbl_44939,from"
re.split(",| ", string)

['from', 'tbl_44939', 'from', 'tbl_44939', 'from']

In [41]:
def find_table(x):
    x = re.split(",| ", x)
    arr = list()
    for i in x:
        if i.startswith("tbl"):
            arr.append(i)
    return arr

In [42]:
"""
Создание столбца table с массивом всех таблиц в запросе
"""
data["table"] = data["query"].apply(find_table)
data.head()

,rn,loguser,query,type_user,table
0,2530,etl_2048,"from tbl_157224,join tbl_157223",1,"[tbl_157224, tbl_157223]"
1,5658,etl_2048,"from tbl_195789,join tbl_195791",1,"[tbl_195789, tbl_195791]"
2,14087,etl_2048,"from tbl_144504,join tbl_144505",1,"[tbl_144504, tbl_144505]"
3,1272,etl_1151,"JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,f...",1,"[tbl_17535, tbl_21652, tbl_26976, tbl_29499, t..."
4,9974,etl_1151,"from tbl_8053,join tbl_9874,join tbl_10303,int...",1,"[tbl_8053, tbl_9874, tbl_10303, tbl_16750, tbl..."


In [43]:
def type_query(x):
    x = x.split(",")
    arr = list()
    for i in x:
        ret = i.split(" ")[0]
        if ret.lower() == "into":
            arr.append(0)
        else:
            arr.append(1)
    return arr

In [44]:
"""
Создания столбца type_query с массивом типов запросов * 1- from, join ; 0 - into*
"""
data["type_query"] = data["query"].apply(type_query)
data.head()

,rn,loguser,query,type_user,table,type_query
0,2530,etl_2048,"from tbl_157224,join tbl_157223",1,"[tbl_157224, tbl_157223]","[1, 1]"
1,5658,etl_2048,"from tbl_195789,join tbl_195791",1,"[tbl_195789, tbl_195791]","[1, 1]"
2,14087,etl_2048,"from tbl_144504,join tbl_144505",1,"[tbl_144504, tbl_144505]","[1, 1]"
3,1272,etl_1151,"JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,f...",1,"[tbl_17535, tbl_21652, tbl_26976, tbl_29499, t...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
4,9974,etl_1151,"from tbl_8053,join tbl_9874,join tbl_10303,int...",1,"[tbl_8053, tbl_9874, tbl_10303, tbl_16750, tbl...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [46]:
data.iloc[3, 2]

'JOIN tbl_17535,join tbl_21652,JOIN tbl_26976,from tbl_29499,FROM tbl_91912,FROM tbl_92053,JOIN tbl_92075,JOIN tbl_92075,JOIN tbl_92100,JOIN tbl_92100,FROM tbl_92114,FROM tbl_92114,FROM tbl_92114,FROM tbl_92114,INTO tbl_338640'

In [45]:
len(data.iloc[3, 4]) == len(data.iloc[3, 5])

True

In [47]:
data.drop(columns=["rn", "query"], inplace=True)
data.head()

,loguser,type_user,table,type_query
0,etl_2048,1,"[tbl_157224, tbl_157223]","[1, 1]"
1,etl_2048,1,"[tbl_195789, tbl_195791]","[1, 1]"
2,etl_2048,1,"[tbl_144504, tbl_144505]","[1, 1]"
3,etl_1151,1,"[tbl_17535, tbl_21652, tbl_26976, tbl_29499, t...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
4,etl_1151,1,"[tbl_8053, tbl_9874, tbl_10303, tbl_16750, tbl...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# new_data = pd.DataFrame(columns=["loguser", "type_user", "table"])
# for i in range(len(data)):
#     for j in data.iloc[i, 2]:
#         new_data.append({"loguser": data.iloc[i, 0], "type_user": data.iloc[i, 1], "table":j}, ignore_index=True)
# new_data.head()

In [48]:
"""
Создане словаря с ключом - название таблицы;
                  заначение - масив 1 элемент - количестово выгрузок пользователями
                                   2 элемент - количество загрузок программистами
"""
dt = dict()
for row in range(len(data)):
    for ind_table in range(len(data.iloc[row, 2])):
        table = data.iloc[row, 2][ind_table]
        if not table in dt:
            dt[table] = [0] * 2
        if data.iloc[row, 3][ind_table] == 0 and data.iloc[row, 1] == 1:
            dt[table][0] +=1
        elif data.iloc[row, 3][ind_table] == 1 and data.iloc[row, 1] == 0:
            dt[table][1] +=1


In [55]:
table = list(dt.keys())
count_etl = list()
count_dev = list()
for tb in dt:
    count_etl.append(dt[tb][0])
    count_dev.append(dt[tb][1])
new_data = pd.DataFrame({"table":table, "count_into_etl": count_etl, "count_select_dev": count_dev})
new_data.head()

,table,count_into_etl,count_select_dev
0,tbl_157224,26,0
1,tbl_157223,0,0
2,tbl_195789,24,0
3,tbl_195791,0,0
4,tbl_144504,24,0


In [56]:
new_data.shape

(48478, 3)

In [57]:
new_data.to_csv("reformed_data.csv", sep=",")

In [61]:
g = pd.read_csv("reformed_data.csv",index_col=0)
g.head()

,table,count_into_etl,count_select_dev
0,tbl_157224,26,0
1,tbl_157223,0,0
2,tbl_195789,24,0
3,tbl_195791,0,0
4,tbl_144504,24,0


In [63]:
"""
Создане словаря с ключом - название таблицы;
                  заначение - масив 1 элемент - количестово выгрузок пользователями
                                    2 элемент - количество выгрузок программистами
                                    3 элемент - количество загрузок пользователем
                                    4 элемент - количество загрузок программистом
                                    5 элемент - количество выгрузок
                                    6 элемент - количество загрузок
"""
dt = dict()
for row in range(len(data)):
    for ind_table in range(len(data.iloc[row, 2])):
        table = data.iloc[row, 2][ind_table]
        if not table in dt:
            dt[table] = [0] * 6
        if data.iloc[row, 3][ind_table] == 0:
            dt[table][4] +=1
            if data.iloc[row, 1] == 1:
                dt[table][0] +=1
            else:
                dt[table][1] +=1
        elif data.iloc[row, 3][ind_table] == 1:
            dt[table][5] +=1
            if data.iloc[row, 1] == 1:
                dt[table][2] +=1
            else:
                dt[table][3] +=1


In [64]:
table = list(dt.keys())
count_down = list()
count_select = list()
count_down_etl = list()
count_select_etl = list()
count_down_dev = list()
count_select_dev = list()
for tb in dt:
    count_down.append(dt[tb][4])
    count_select.append(dt[tb][5])
    count_down_etl.append(dt[tb][0])
    count_select_etl.append(dt[tb][2])
    count_down_dev.append(dt[tb][1])
    count_select_dev.append(dt[tb][3])
new_data = pd.DataFrame({"table":table, 
                         "count_all_into": count_down, 
                         "count_all_select": count_select,
                        "count_etl_into": count_down_etl,
                        "count_etl_select":count_select_etl,
                        "count_dev_into":count_down_dev,
                        "count_dev_select":count_select_dev})
new_data.head()

,table,count_all_into,count_all_select,count_etl_into,count_etl_select,count_dev_into,count_dev_select
0,tbl_157224,26,128,26,128,0,0
1,tbl_157223,0,65,0,65,0,0
2,tbl_195789,24,116,24,116,0,0
3,tbl_195791,0,60,0,60,0,0
4,tbl_144504,24,130,24,130,0,0


In [65]:
new_data.to_csv("reformed_data_all.csv", sep=",")